In [3]:
import numpy as np
import pandas as pd
import urllib.request as urllib
from bs4 import BeautifulSoup
import time
import datetime
from dateutil import parser

In [4]:
biig_html = urllib.urlopen("http://ufcstats.com/statistics/events/completed?page=all")
biig_soup = BeautifulSoup(biig_html.read())
linkss = biig_soup.find_all("a", "b-link")[1:]
linkss = [l.get("href") for l in linkss]
dates = []
all_results = []
all_fighters = []
all_methods = []
all_times = []
all_stats = []

for l1 in linkss:
    event_html = urllib.urlopen(l1)
    event_soup = BeautifulSoup(event_html.read())

    date = event_soup.find(class_="b-list__box-list-item").get_text().strip().split("\n")[2].strip()
    date = parser.parse(date)
    dates.append(date)
    #now I'll get all the links to the fights in the event
    link_soup = event_soup.find_all("a", "b-flag")
    links = [l.get("href") for l in link_soup]
    links = list(set(links))

    for l in links:
        html = urllib.urlopen(l)
        soup = BeautifulSoup(html.read())

        fighters_soup = soup.find_all(class_="b-link b-fight-details__person-link")
        fighter1 = fighters_soup[0].get_text().strip()
        fighter2 = fighters_soup[1].get_text().strip()
        all_fighters.append(fighter1)
        all_fighters.append(fighter2)

        results_soup = soup.find_all(class_="b-fight-details__person-status")
        result1 = results_soup[0].get_text().strip()
        result2 = "L"
        if result1 != "W":
            result2 = result1
        all_results.append(result1)
        all_results.append(result2)

        details_soup = soup.find(class_="b-fight-details__text")
        method = details_soup.find(style="font-style: normal").get_text().strip()
        all_methods.append(method)

        time_soup = details_soup.find_all(class_="b-fight-details__text-item")
        round = time_soup[0].get_text().strip()[-1]
        end_time = time_soup[1].get_text().strip()
        end_time = end_time.split("\n")[-1].strip()

        #fight duration in seconds
        end_time = time.strptime(end_time.split(',')[0],'%M:%S')
        end_time = datetime.timedelta(hours=end_time.tm_hour,minutes=end_time.tm_min,seconds=end_time.tm_sec).total_seconds()
        end_time = (((int(round)-1) * 300) + end_time)
        all_times.append(end_time)

        tables = soup.find_all("tr")
        totals_soup = tables[0:2]
        totals_stats_soup = totals_soup[0].get_text().strip().split("\n")
        totals_stats = [s.strip() for s in totals_stats_soup]
        totals_stats = list(filter(lambda a: a != "", totals_stats))
        totals_stats_labels = totals_stats

        fighter_totals_soup = totals_soup[1]
        fighters_totals_stats = fighter_totals_soup.get_text().strip().split("\n")
        fighters_totals_stats = [s.strip() for s in fighters_totals_stats]
        fighters_totals_stats = list(filter(lambda a: a != "", fighters_totals_stats))
        fighters_totals_stats



        fighter1_totals_stats = [fighters_totals_stats[i] for i in range(0, len(fighters_totals_stats), 2)]
        fighter2_totals_stats = [fighters_totals_stats[i + 1] for i in range(0, len(fighters_totals_stats), 2)]
        
        fighter1_totals_dict = {totals_stats_labels[i]: fighter1_totals_stats[i] for i in range(0, len(totals_stats_labels))}
        fighter2_totals_dict = {totals_stats_labels[i]: fighter2_totals_stats[i] for i in range(0, len(totals_stats_labels))}


        strike_soup = soup.find_all("table")[2]
        strike_labels = strike_soup.find(class_="b-fight-details__table-head")
        strike_labels = strike_labels.get_text().strip().split("\n")
        strike_labels = [s.strip() for s in strike_labels]
        strike_labels = list(filter(lambda a: a != "", strike_labels))
        strike_labels

        strike_details = strike_soup.find_all("p")
        strike_details = [s.get_text().strip() for s in strike_details]

        fighter1_strike_details = [strike_details[i] for i in range(0, len(strike_details), 2)]
        fighter2_strike_details = [strike_details[i + 1] for i in range(0, len(strike_details), 2)]

        fighter1_strikes_dict = {strike_labels[i]: fighter1_strike_details[i] for i in range(0, len(strike_labels))}
        fighter2_strikes_dict = {strike_labels[i]: fighter2_strike_details[i] for i in range(0, len(strike_labels))}


        #merging the fight stat dicts
        fighter1stats = {**fighter1_strikes_dict, **fighter1_totals_dict}
        fighter2stats = {**fighter2_strikes_dict, **fighter2_totals_dict}

        all_stats.append(fighter1stats)
        all_stats.append(fighter2stats)
all_stats


IndexError: list index out of range

In [5]:
len(all_stats)

11948

In [6]:
"""
we can maybe keep the rest into a dataframe and pickle it
"""
fight_df = pd.DataFrame()
fight_df["result"] = all_results
fight_df["fighter"] = all_fighters

In [7]:
#looks like all the fight stats did not scrape perfectly, we need to find out who 
#has missing data
for i in range(0, len(all_stats)):
    if all_fighters[i] == all_stats[i]["Fighter"]:
        print(i)
        print(all_fighters[i])
        print(all_stats[i]["Fighter"])

0
Lauren Murphy
Lauren Murphy
1
Joanne Calderwood
Joanne Calderwood
2
Matt Frevola
Matt Frevola
3
Terrance McKinney
Terrance McKinney
4
Eryk Anders
Eryk Anders
5
Darren Stewart
Darren Stewart
6
Demian Maia
Demian Maia
7
Belal Muhammad
Belal Muhammad
8
Paul Craig
Paul Craig
9
Jamahal Hill
Jamahal Hill
10
Chase Hooper
Chase Hooper
11
Steven Peterson
Steven Peterson
12
Drew Dober
Drew Dober
13
Brad Riddell
Brad Riddell
14
Israel Adesanya
Israel Adesanya
15
Marvin Vettori
Marvin Vettori
16
Deiveson Figueiredo
Deiveson Figueiredo
17
Brandon Moreno
Brandon Moreno
18
Leon Edwards
Leon Edwards
19
Nate Diaz
Nate Diaz
20
Movsar Evloev
Movsar Evloev
21
Hakeem Dawodu
Hakeem Dawodu
22
Pannie Kianzad
Pannie Kianzad
23
Alexis Davis
Alexis Davis
24
Carlos Felipe
Carlos Felipe
25
Jake Collier
Jake Collier
26
Fares Ziam
Fares Ziam
27
Luigi Vendramini
Luigi Vendramini
28
Roman Dolidze
Roman Dolidze
29
Laureano Staropoli
Laureano Staropoli
30
Makwan Amirkhani
Makwan Amirkhani
31
Kamuela Kirk
Kamuela Kirk


Miranda Maverick
646
Alex Oliveira
Alex Oliveira
647
Shavkat Rakhmonov
Shavkat Rakhmonov
648
Khabib Nurmagomedov
Khabib Nurmagomedov
649
Justin Gaethje
Justin Gaethje
650
Robert Whittaker
Robert Whittaker
651
Jared Cannonier
Jared Cannonier
652
Jacob Malkoun
Jacob Malkoun
653
Phil Hawes
Phil Hawes
654
Gadzhimurad Antigulov
Gadzhimurad Antigulov
655
Maxim Grishin
Maxim Grishin
656
Claudio Silva
Claudio Silva
657
James Krause
James Krause
658
Said Nurmagomedov
Said Nurmagomedov
659
Mark Striegl
Mark Striegl
660
Jamie Mullarkey
Jamie Mullarkey
661
Fares Ziam
Fares Ziam
662
Mateusz Gamrot
Mateusz Gamrot
663
Guram Kutateladze
Guram Kutateladze
664
Junyong Park
Junyong Park
665
John Phillips
John Phillips
666
Katlyn Chookagian
Katlyn Chookagian
667
Jessica Andrade
Jessica Andrade
668
Brian Ortega
Brian Ortega
669
Chan Sung Jung
Chan Sung Jung
670
Jimmy Crute
Jimmy Crute
671
Modestas Bukauskas
Modestas Bukauskas
672
Gillian Robertson
Gillian Robertson
673
Poliana Botelho
Poliana Botelho
674
T

Mara Romero Borella
Mara Romero Borella
1146
Chase Sherman
Chase Sherman
1147
Ike Villanueva
Ike Villanueva
1148
Alexander Hernandez
Alexander Hernandez
1149
Drew Dober
Drew Dober
1150
Hunter Azure
Hunter Azure
1151
Brian Kelleher
Brian Kelleher
1152
Anthony Smith
Anthony Smith
1153
Glover Teixeira
Glover Teixeira
1154
Ben Rothwell
Ben Rothwell
1155
Ovince Saint Preux
Ovince Saint Preux
1156
Sijara Eubanks
Sijara Eubanks
1157
Sarah Moras
Sarah Moras
1158
Ricky Simon
Ricky Simon
1159
Ray Borg
Ray Borg
1160
Michael Johnson
Michael Johnson
1161
Thiago Moises
Thiago Moises
1162
Andrei Arlovski
Andrei Arlovski
1163
Philipe Lins
Philipe Lins
1164
Gabriel Benitez
Gabriel Benitez
1165
Omar Morales
Omar Morales
1166
Carla Esparza
Carla Esparza
1167
Michelle Waterson
Michelle Waterson
1168
Jeremy Stephens
Jeremy Stephens
1169
Calvin Kattar
Calvin Kattar
1170
Aleksei Oleinik
Aleksei Oleinik
1171
Fabricio Werdum
Fabricio Werdum
1172
Henry Cejudo
Henry Cejudo
1173
Dominick Cruz
Dominick Cruz
1174
F

1812
Claudio Silva
Claudio Silva
1813
Cole Williams
Cole Williams
1814
Trevin Giles
Trevin Giles
1815
Gerald Meerschaert
Gerald Meerschaert
1816
Antonina Shevchenko
Antonina Shevchenko
1817
Lucie Pudilova
Lucie Pudilova
1818
Miranda Granger
Miranda Granger
1819
Hannah Goldy
Hannah Goldy
1820
Jim Miller
Jim Miller
1821
Clay Guida
Clay Guida
1822
Hakeem Dawodu
Hakeem Dawodu
1823
Yoshinori Horie
Yoshinori Horie
1824
Alexandre Pantoja
Alexandre Pantoja
1825
Deiveson Figueiredo
Deiveson Figueiredo
1826
Cristiane Justino
Cristiane Justino
1827
Felicia Spencer
Felicia Spencer
1828
Olivier Aubin-Mercier
Olivier Aubin-Mercier
1829
Arman Tsarukyan
Arman Tsarukyan
1830
Max Holloway
Max Holloway
1831
Frankie Edgar
Frankie Edgar
1832
Marc-Andre Barriault
Marc-Andre Barriault
1833
Krzysztof Jotko
Krzysztof Jotko
1834
Gavin Tucker
Gavin Tucker
1835
SeungWoo Choi
SeungWoo Choi
1836
Alexis Davis
Alexis Davis
1837
Viviane Araujo
Viviane Araujo
1838
Erik Koch
Erik Koch
1839
Kyle Stewart
Kyle Stewart
1840

Olivier Aubin-Mercier
Olivier Aubin-Mercier
2479
Gilbert Burns
Gilbert Burns
2480
Valentina Shevchenko
Valentina Shevchenko
2481
Joanna Jedrzejczyk
Joanna Jedrzejczyk
2482
Chad Laprise
Chad Laprise
2483
Dhiego Lima
Dhiego Lima
2484
Devin Clark
Devin Clark
2485
Aleksandar Rakic
Aleksandar Rakic
2486
Junior Dos Santos
Junior Dos Santos
2487
Tai Tuivasa
Tai Tuivasa
2488
Jimmy Crute
Jimmy Crute
2489
Paul Craig
Paul Craig
2490
Keita Nakamura
Keita Nakamura
2491
Salim Touahri
Salim Touahri
2492
Alex Gorgees
Alex Gorgees
2493
Damir Ismagulov
Damir Ismagulov
2494
Jake Matthews
Jake Matthews
2495
Anthony Rocco Martin
Anthony Rocco Martin
2496
Wilson Reis
Wilson Reis
2497
Ben Nguyen
Ben Nguyen
2498
Yushin Okami
Yushin Okami
2499
Aleksei Kunchenko
Aleksei Kunchenko
2500
Mark Hunt
Mark Hunt
2501
Justin Willis
Justin Willis
2502
Mauricio Rua
Mauricio Rua
2503
Tyson Pedro
Tyson Pedro
2504
Suman Mokhtarian
Suman Mokhtarian
2505
Sodiq Yusuff
Sodiq Yusuff
2506
Elias Garcia
Elias Garcia
2507
Kai Kara-Fr

...

Dennis Siver
8977
Matt Wiman
Matt Wiman
8978
Carlos Condit
Carlos Condit
8979
Dong Hyun Kim
Dong Hyun Kim
8980
Tito Ortiz
Tito Ortiz
8981
Ryan Bader
Ryan Bader
8982
Jeff Hougland
Jeff Hougland
8983
Donny Walker
Donny Walker
8984
Brian Bowles
Brian Bowles
8985
Takeya Mizugaki
Takeya Mizugaki
8986
Wanderlei Silva
Wanderlei Silva
8987
Chris Leben
Chris Leben
8988
Melvin Guillard
Melvin Guillard
8989
Shane Roller
Shane Roller
8990
Anthony Njokuani
Anthony Njokuani
8991
Andre Winner
Andre Winner
8992
Ricardo Lamas
Ricardo Lamas
8993
Matt Grice
Matt Grice
8994
Tyson Griffin
Tyson Griffin
8995
Manvel Gamburyan
Manvel Gamburyan
8996
Joe Stevenson
Joe Stevenson
8997
Javier Vazquez
Javier Vazquez
8998
Joe Lauzon
Joe Lauzon
8999
Curt Warburton
Curt Warburton
9000
Daniel Roberts
Daniel Roberts
9001
Rich Attonito
Rich Attonito
9002
Matt Brown
Matt Brown
9003
John Howard
John Howard
9004
Nik Lentz
Nik Lentz
9005
Charles Oliveira
Charles Oliveira
9006
Michael Johnson
Michael Johnson
9007
Edward Faalo

Rafaello Oliveira
Rafaello Oliveira
9644
Rob Emerson
Rob Emerson
9645
Nik Lentz
Nik Lentz
9646
Nate Diaz
Nate Diaz
9647
Rory Markham
Rory Markham
9648
Georges St-Pierre
Georges St-Pierre
9649
Dan Hardy
Dan Hardy
9650
Rodney Wallace
Rodney Wallace
9651
Jared Hamman
Jared Hamman
9652
Kurt Pellegrino
Kurt Pellegrino
9653
Fabricio Camoes
Fabricio Camoes
9654
Rousimar Palhares
Rousimar Palhares
9655
Tomasz Drwal
Tomasz Drwal
9656
Ricardo Almeida
Ricardo Almeida
9657
Matt Brown
Matt Brown
9658
Matthew Riddle
Matthew Riddle
9659
Greg Soto
Greg Soto
9660
Frank Mir
Frank Mir
9661
Shane Carwin
Shane Carwin
9662
Jim Miller
Jim Miller
9663
Mark Bocek
Mark Bocek
9664
Ben Saunders
Ben Saunders
9665
Jon Fitch
Jon Fitch
9666
Clay Guida
Clay Guida
9667
Shannon Gugerty
Shannon Gugerty
9668
Duane Ludwig
Duane Ludwig
9669
Darren Elkins
Darren Elkins
9670
Junior Dos Santos
Junior Dos Santos
9671
Gabriel Gonzaga
Gabriel Gonzaga
9672
Eliot Marshall
Eliot Marshall
9673
Vladimir Matyushenko
Vladimir Matyushenk

Paul Kelly
10310
Dan Hardy
Dan Hardy
10311
Akihiro Gono
Akihiro Gono
10312
Keith Jardine
Keith Jardine
10313
Brandon Vera
Brandon Vera
10314
Shane Carwin
Shane Carwin
10315
Neil Wain
Neil Wain
10316
Michael Bisping
Michael Bisping
10317
Chris Leben
Chris Leben
10318
Luiz Cane
Luiz Cane
10319
Rameau Thierry Sokoudjou
Rameau Thierry Sokoudjou
10320
Mike Massenzio
Mike Massenzio
10321
Drew McFedries
Drew McFedries
10322
Clay Guida
Clay Guida
10323
Mac Danzig
Mac Danzig
10324
Alan Belcher
Alan Belcher
10325
Ed Herman
Ed Herman
10326
Wilson Gouveia
Wilson Gouveia
10327
Ryan Jensen
Ryan Jensen
10328
Jason Brilz
Jason Brilz
10329
Brad Morris
Brad Morris
10330
Nate Diaz
Nate Diaz
10331
Josh Neer
Josh Neer
10332
Dan Miller
Dan Miller
10333
Rob Kimmons
Rob Kimmons
10334
Joe Lauzon
Joe Lauzon
10335
Kyle Bradley
Kyle Bradley
10336
Alessio Sakara
Alessio Sakara
10337
Joe Vedepo
Joe Vedepo
10338
Eric Schafer
Eric Schafer
10339
Houston Alexander
Houston Alexander
10340
Rich Franklin
Rich Franklin
103

Josh Koscheck
Josh Koscheck
10977
Jeff Joslin
Jeff Joslin
10978
Alan Belcher
Alan Belcher
10979
Jorge Santiago
Jorge Santiago
10980
Antoni Hardonk
Antoni Hardonk
10981
Sherman Pendergarst
Sherman Pendergarst
10982
Brandon Vera
Brandon Vera
10983
Frank Mir
Frank Mir
10984
Tim Sylvia
Tim Sylvia
10985
Jeff Monson
Jeff Monson
10986
Drew McFedries
Drew McFedries
10987
Alessio Sakara
Alessio Sakara
10988
Nick Diaz
Nick Diaz
10989
Gleison Tibau
Gleison Tibau
10990
Jake O'Brien
Jake O'Brien
10991
Josh Shockman
Josh Shockman
10992
Georges St-Pierre
Georges St-Pierre
10993
Matt Hughes
Matt Hughes
10994
Joe Stevenson
Joe Stevenson
10995
Dokonjonosuke Mishima
Dokonjonosuke Mishima
10996
James Irvin
James Irvin
10997
Hector Ramirez
Hector Ramirez
10998
Martin Kampmann
Martin Kampmann
10999
Thales Leites
Thales Leites
11000
Din Thomas
Din Thomas
11001
Rich Clementi
Rich Clementi
11002
Charles McCarthy
Charles McCarthy
11003
Gideon Ray
Gideon Ray
11004
Matt Serra
Matt Serra
11005
Chris Lytle
Chris Ly

11630
Benji Radach
Benji Radach
11631
Nick Serra
Nick Serra
11632
Tony Fryklund
Tony Fryklund
11633
Rodrigo Ruas
Rodrigo Ruas
11634
Pete Spratt
Pete Spratt
11635
Zach Light
Zach Light
11636
Yves Edwards
Yves Edwards
11637
Joao Pierini
Joao Pierini
11638
Robbie Lawler
Robbie Lawler
11639
Aaron Riley
Aaron Riley
11640
Benji Radach
Benji Radach
11641
Steve Berger
Steve Berger
11642
Ricco Rodriguez
Ricco Rodriguez
11643
Tsuyoshi Kohsaka
Tsuyoshi Kohsaka
11644
Murilo Bustamante
Murilo Bustamante
11645
Matt Lindland
Matt Lindland
11646
Phil Baroni
Phil Baroni
11647
Amar Suloev
Amar Suloev
11648
Caol Uno
Caol Uno
11649
Yves Edwards
Yves Edwards
11650
BJ Penn
BJ Penn
11651
Paul Creighton
Paul Creighton
11652
Ivan Salaverry
Ivan Salaverry
11653
Andrei Semenov
Andrei Semenov
11654
Matt Lindland
Matt Lindland
11655
Pat Miletich
Pat Miletich
11656
Frank Mir
Frank Mir
11657
Pete Williams
Pete Williams
11658
Sean Sherk
Sean Sherk
11659
Jutaro Nakao
Jutaro Nakao
11660
Matt Serra
Matt Serra
11661
Kell

In [8]:
all_fighters = all_fighters[:-2]

In [9]:
all_results = all_results[:-4]

In [10]:
all_fighters = all_fighters[:-2]
all_stats = all_stats[:-2]

In [26]:
all_results2 = all_results + ["W", "L"]

In [20]:
""" 
now dates, the hard one, made a mistake while scraping, shoudlve
repeated the same date for every fight, may be faster to rescrape
"""
biig_html = urllib.urlopen("http://ufcstats.com/statistics/events/completed?page=all")
biig_soup = BeautifulSoup(biig_html.read())
linkss = biig_soup.find_all("a", "b-link")[1:]
linkss = [l.get("href") for l in linkss]
dates = []
for l1 in linkss:
    event_html = urllib.urlopen(l1)
    event_soup = BeautifulSoup(event_html.read())

    date = event_soup.find(class_="b-list__box-list-item").get_text().strip().split("\n")[2].strip()
    date = parser.parse(date)
    #now I'll get all the links to the fights in the event
    link_soup = event_soup.find_all("a", "b-flag")
    links = [l.get("href") for l in link_soup]
    links = list(set(links))

    for l in links:
        html = urllib.urlopen(l)
        soup = BeautifulSoup(html.read())
        dates.append(date)

In [27]:
len(all_methods)
all_methods_to_use = all_methods[:-2]
len(all_methods_to_use)

5973

In [28]:
len(dates)
dates_to_use = dates[:5973]
len(dates_to_use)

5973

In [29]:
len(all_times)
all_times_to_use = all_times[:-2]
len(all_times_to_use)

5973

In [26]:
len(all_fighters) / 2
    

5973.0

In [31]:
fighter_ones = []
fighter_twos = []
for i in range(5973):
    fighter_ones.append(all_fighters[i])
    fighter_twos.append(all_fighters[i + 1])
    

In [32]:
len(fighter_ones)

5973

In [33]:
len(fighter_twos)

5973

In [34]:
df = pd.DataFrame()

In [35]:
df['fighter_1'] = fighter_ones
df

,fighter_1
0,Lauren Murphy
1,Joanne Calderwood
2,Matt Frevola
3,Terrance McKinney
4,Eryk Anders
...,...
5968,Sergio Moraes
5969,Mickael Lebout
5970,Joanne Calderwood
5971,Maryna Moroz


In [36]:
df['fighter_2'] = fighter_twos
df

,fighter_1,fighter_2
0,Lauren Murphy,Joanne Calderwood
1,Joanne Calderwood,Matt Frevola
2,Matt Frevola,Terrance McKinney
3,Terrance McKinney,Eryk Anders
4,Eryk Anders,Darren Stewart
...,...,...
5968,Sergio Moraes,Mickael Lebout
5969,Mickael Lebout,Joanne Calderwood
5970,Joanne Calderwood,Maryna Moroz
5971,Maryna Moroz,Daniel Omielanczuk


In [37]:
df['date'] = dates_to_use

In [41]:
df['method'] = all_methods_to_use
df

,fighter_1,fighter_2,date,method
0,Lauren Murphy,Joanne Calderwood,2021-06-12,Decision - Split
1,Joanne Calderwood,Matt Frevola,2021-06-12,KO/TKO
2,Matt Frevola,Terrance McKinney,2021-06-12,Decision - Unanimous
3,Terrance McKinney,Eryk Anders,2021-06-12,Decision - Unanimous
4,Eryk Anders,Darren Stewart,2021-06-12,KO/TKO
...,...,...,...,...
5968,Sergio Moraes,Mickael Lebout,1999-01-08,Decision - Unanimous
5969,Mickael Lebout,Joanne Calderwood,1999-01-08,TKO - Doctor's Stoppage
5970,Joanne Calderwood,Maryna Moroz,1998-10-16,Decision - Split
5971,Maryna Moroz,Daniel Omielanczuk,1998-10-16,KO/TKO


In [42]:
df['duration'] = all_times_to_use
df

,fighter_1,fighter_2,date,method,duration
0,Lauren Murphy,Joanne Calderwood,2021-06-12,Decision - Split,900.0
1,Joanne Calderwood,Matt Frevola,2021-06-12,KO/TKO,7.0
2,Matt Frevola,Terrance McKinney,2021-06-12,Decision - Unanimous,900.0
3,Terrance McKinney,Eryk Anders,2021-06-12,Decision - Unanimous,900.0
4,Eryk Anders,Darren Stewart,2021-06-12,KO/TKO,119.0
...,...,...,...,...,...
5968,Sergio Moraes,Mickael Lebout,1999-01-08,Decision - Unanimous,480.0
5969,Mickael Lebout,Joanne Calderwood,1999-01-08,TKO - Doctor's Stoppage,451.0
5970,Joanne Calderwood,Maryna Moroz,1998-10-16,Decision - Split,780.0
5971,Maryna Moroz,Daniel Omielanczuk,1998-10-16,KO/TKO,487.0


In [43]:
df.to_pickle("method_duration_data.pkl")

In [44]:
df

,fighter_1,fighter_2,date,method,duration
0,Lauren Murphy,Joanne Calderwood,2021-06-12,Decision - Split,900.0
1,Joanne Calderwood,Matt Frevola,2021-06-12,KO/TKO,7.0
2,Matt Frevola,Terrance McKinney,2021-06-12,Decision - Unanimous,900.0
3,Terrance McKinney,Eryk Anders,2021-06-12,Decision - Unanimous,900.0
4,Eryk Anders,Darren Stewart,2021-06-12,KO/TKO,119.0
...,...,...,...,...,...
5968,Sergio Moraes,Mickael Lebout,1999-01-08,Decision - Unanimous,480.0
5969,Mickael Lebout,Joanne Calderwood,1999-01-08,TKO - Doctor's Stoppage,451.0
5970,Joanne Calderwood,Maryna Moroz,1998-10-16,Decision - Split,780.0
5971,Maryna Moroz,Daniel Omielanczuk,1998-10-16,KO/TKO,487.0
